In [1]:
import pandas as pd
from sktime.base import load

In [2]:
clf = load('model')
data = pd.read_parquet('test.parquet')
data

,id,dates,values
0,6125,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[1.85, -0.04, 0.19, -0.45, -0.75, -0.95, -2.91..."
1,26781,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[-0.41, 0.39, -0.47, -0.9, -1.46, -0.51, 0.51,..."
2,13333,"[2016-06-01, 2016-07-01, 2016-08-01, 2016-09-0...","[-0.29, -1.26, 0.17, -1.22, 0.45, -0.94, 0.16,..."
3,53218,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[-1.47, 1.55, -0.03, 0.57, -0.57, 0.6, 0.27, 1..."
4,84204,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[2.33, 1.39, -1.03, -2.64, 1.89, 1.77, 1.43, 1..."
...,...,...,...
19995,80341,"[2016-03-01, 2016-04-01, 2016-05-01, 2016-06-0...","[3.01, -0.58, 1.55, 0.48, -0.35, 1.93, 3.86, 2..."
19996,5891,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[0.66, 1.3, 2.78, -0.25, -1.97, -0.55, -1.08, ..."
19997,29091,"[2017-01-01, 2017-02-01, 2017-03-01, 2017-04-0...","[0.09, 0.44, 1.55, 0.15, 0.3, 0.19, 0.34, 1.05..."
19998,85877,"[2017-04-01, 2017-05-01, 2017-06-01, 2017-07-0...","[0.28, 1.92, 1.14, 2.4, 1.46, 1.08, -0.12, 0.6..."


In [3]:
df = data.explode(['dates', 'values']).dropna()
df['dates'] = df['dates'].apply(lambda x: pd.Timestamp(x))
df.index = pd.MultiIndex.from_frame(df[['id', 'dates']])
df['values'] = df['values'].astype(float)
df = df.drop(labels=['id', 'dates'], axis=1)
df

values
id    dates             
6125  2016-01-01    1.85
      2016-02-01   -0.04
      2016-03-01    0.19
      2016-04-01   -0.45
      2016-05-01   -0.75
...                  ...
73528 2023-02-01   -0.53
      2023-03-01    1.65
      2023-04-01    0.07
      2023-05-01    0.62
      2023-06-01    0.86

[1285873 rows x 1 columns]

In [4]:
pred = clf.predict(df)

In [5]:
res = pd.DataFrame({'score' : pred}, index=df.index.get_level_values(0).drop_duplicates())
res.to_csv('submission.csv')